# Prevendo fraudes de cartão de crédito

A fraude de cartão de crédito é uma das principais causas de roubo de identidade em todo o mundo. As instituições financeiras empregam uma grande variedade de técnicas diferentes para prevenir fraudes, sendo uma das mais comuns a Regressão Logística.

Neste projeto, você é um Cientista de Dados que trabalha para uma empresa de cartão de crédito. Você tem acesso a um conjunto de dados, que representa um conjunto típico de transações de cartão de crédito. Sua tarefa é usar a regressão logística e criar um modelo preditivo para determinar se uma transação é fraudulenta ou não.

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Carregar os dados

**1.** O arquivo `Transações.csv` contém dados de 200 mil transações de cartão de crédito simuladas. Comece carregando os dados em um DataFrame pandas chamado `transacoes`. Imprima algumas linhas e a contagem total de linhas. 

In [2]:
transacoes = pd.read_csv('Transações.csv')

transacoes.head(10)


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,8,CASH_OUT,158007.12,C424875646,0.00,0.00,C1298177219,474016.32,1618631.97,0
1,236,CASH_OUT,457948.30,C1342616552,0.00,0.00,C1323169990,2720411.37,3178359.67,0
2,37,CASH_IN,153602.99,C900876541,11160428.67,11314031.67,C608741097,3274930.56,3121327.56,0
3,331,CASH_OUT,49555.14,C177696810,10865.00,0.00,C462716348,0.00,49555.14,0
4,250,CASH_OUT,29648.02,C788941490,0.00,0.00,C1971700992,56933.09,86581.10,0
5,182,PAYMENT,15712.66,C365217190,13981.00,0.00,M1108542644,0.00,0.00,0
6,355,PAYMENT,357.15,C1752574405,283349.71,282992.56,M807801422,0.00,0.00,0
7,302,PAYMENT,19477.23,C1681583142,0.00,0.00,M911688965,0.00,0.00,0
8,35,PAYMENT,56554.20,C643126692,503065.44,446511.24,M1453972991,0.00,0.00,0
9,156,CASH_OUT,180660.51,C106721885,10044.00,0.00,C1944402764,32319.16,212979.67,0


In [3]:
transacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   step            199999 non-null  int64  
 1   type            199999 non-null  object 
 2   amount          199999 non-null  float64
 3   nameOrig        199999 non-null  object 
 4   oldbalanceOrg   199999 non-null  float64
 5   newbalanceOrig  199999 non-null  float64
 6   nameDest        199999 non-null  object 
 7   oldbalanceDest  199999 non-null  float64
 8   newbalanceDest  199999 non-null  float64
 9   isFraud         199999 non-null  int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 15.3+ MB


### Limpe os dados

**2.** Olhando para o conjunto de dados, combinado com nosso conhecimento de transações de cartão de crédito em geral, podemos ver que existem algumas colunas interessantes para serem observadas. Sabemos que a coluna `amount`(quantia) de uma determinada transação será importante. Calcule estatísticas de resumo para esta coluna. Como é a distribuição?

In [4]:
transacoes.amount.describe()

count    1.999990e+05
mean     1.802425e+05
std      6.255482e+05
min      0.000000e+00
25%      1.338746e+04
50%      7.426695e+04
75%      2.086376e+05
max      5.204280e+07
Name: amount, dtype: float64

**3.** Temos muitas informações sobre o tipo (`type`) de transação que estamos analisando. Crie uma nova coluna chamada `é_Pagamento` que atribui um `1` quando `type` é “PAYMENT” ou “DEBIT” e `0` caso de outro tipo.

In [5]:
transacoes['ehPagamento'] = transacoes.apply(lambda x: 1 if x['type'] == 'PAYMENT' or x['type'] == 'DEBIT' else 0, axis = 1)

transacoes.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,ehPagamento
0,8,CASH_OUT,158007.12,C424875646,0.00,0.00,C1298177219,474016.32,1618631.97,0,0
1,236,CASH_OUT,457948.30,C1342616552,0.00,0.00,C1323169990,2720411.37,3178359.67,0,0
2,37,CASH_IN,153602.99,C900876541,11160428.67,11314031.67,C608741097,3274930.56,3121327.56,0,0
3,331,CASH_OUT,49555.14,C177696810,10865.00,0.00,C462716348,0.00,49555.14,0,0
4,250,CASH_OUT,29648.02,C788941490,0.00,0.00,C1971700992,56933.09,86581.10,0,0
5,182,PAYMENT,15712.66,C365217190,13981.00,0.00,M1108542644,0.00,0.00,0,1
6,355,PAYMENT,357.15,C1752574405,283349.71,282992.56,M807801422,0.00,0.00,0,1
7,302,PAYMENT,19477.23,C1681583142,0.00,0.00,M911688965,0.00,0.00,0,1
8,35,PAYMENT,56554.20,C643126692,503065.44,446511.24,M1453972991,0.00,0.00,0,1
9,156,CASH_OUT,180660.51,C106721885,10044.00,0.00,C1944402764,32319.16,212979.67,0,0


**4.** Da mesma forma, crie uma coluna chamada `é_movimentação`, que capturará se o dinheiro saiu da conta de origem. Esta coluna terá um valor de 1 quando `type` for “CASH_OUT” ou “TRANSFER”, e 0 caso contrário.

In [6]:
transacoes['ehMovimentacao'] = transacoes.apply(lambda x: 1 if x['type'] == 'CASH_OUT' or x['type'] == 'TRANSFER' else 0, axis = 1)

transacoes.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,ehPagamento,ehMovimentacao
0,8,CASH_OUT,158007.12,C424875646,0.00,0.00,C1298177219,474016.32,1618631.97,0,0,1
1,236,CASH_OUT,457948.30,C1342616552,0.00,0.00,C1323169990,2720411.37,3178359.67,0,0,1
2,37,CASH_IN,153602.99,C900876541,11160428.67,11314031.67,C608741097,3274930.56,3121327.56,0,0,0
3,331,CASH_OUT,49555.14,C177696810,10865.00,0.00,C462716348,0.00,49555.14,0,0,1
4,250,CASH_OUT,29648.02,C788941490,0.00,0.00,C1971700992,56933.09,86581.10,0,0,1
5,182,PAYMENT,15712.66,C365217190,13981.00,0.00,M1108542644,0.00,0.00,0,1,0
6,355,PAYMENT,357.15,C1752574405,283349.71,282992.56,M807801422,0.00,0.00,0,1,0
7,302,PAYMENT,19477.23,C1681583142,0.00,0.00,M911688965,0.00,0.00,0,1,0
8,35,PAYMENT,56554.20,C643126692,503065.44,446511.24,M1453972991,0.00,0.00,0,1,0
9,156,CASH_OUT,180660.51,C106721885,10044.00,0.00,C1944402764,32319.16,212979.67,0,0,1


**5.** Em fraudes financeiras, outro fator fundamental a ser investigado seria a diferença de valor entre a conta de origem e a conta de destino. A hipótese, neste caso, é que contas de destino com valor significativamente diferente podem ser suspeitas de fraude. Vamos criar uma coluna chamada `diferençaContas` com a diferença absoluta das colunas `oldbalanceOrge` e `oldbalanceDest`

In [7]:
transacoes['diferencaContas'] = transacoes.apply(lambda x: abs(x['oldbalanceOrg'] - x['oldbalanceDest']), axis = 1)

### Selecionar e dividir os dados

**6.** Antes de começarmos a treinar nosso modelo, precisamos definir nossos recursos (variáveis de entrada) e a coluna que será a saída. A saída neste conjunto de dados é a coluna `isFraud`. Crie uma variável chamada `x` que será um array composto pelos seguintes campos:

- `amount`
- `é_Pagamento`
- `é_movimentação`
- `diferençaContas`

Crie também uma variável chamada `y` com a coluna `isFraud`.

In [8]:
x = transacoes[['amount', 'ehPagamento', 'ehMovimentacao', 'diferencaContas']]
y = transacoes[['isFraud']]

**7.** Divida os dados em conjuntos de treinamento e teste usando o método `train_test_split()` do `sklearn`. Usaremos o conjunto de treinamento para treinar o modelo e o conjunto de teste para avaliar o modelo. Use um `test_size` de 0.3.

In [9]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.3)

### Normalize os dados

**8.** No `sklearn` a implementação da regressão logística utiliza regularização e portanto precisamos dimencionar nossas variáveis de entrada. Crie um objeto `StandardScaler` , Utilize `.fit_transform()` nas variáveis de entrada do conjunto de treinamento e `.transform()` nas variáveis de entrada do conjunto de de teste.

In [10]:
ss = StandardScaler()

x_treino = ss.fit_transform(x_treino)

x_teste = ss.transform(x_teste)


### Criar e avaliar o modelo

**9.** Crie um modelo de regressão logística no `sklearn`.

In [11]:
regresaologistica = LogisticRegression()

regresaologistica.fit(x_treino, y_treino)

c:\Users\rafin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

**10.** Execute o método `.score()` do modelo nos dados de treinamento e imprima a pontuação de treinamento.

A pontuação do modelo nos dados de treinamento processará os dados de treinamento por meio do modelo treinado e preverá quais transações são fraudulentas. A pontuação retornada é a porcentagem de classificações corretas ou a precisão.

In [12]:
regresaologistica.score(x_treino, y_treino)

0.998571418367274

**11.** Imprima a matriz de confusão do modelo utilizando os dados do conjunto de teste.

In [13]:
y_pred = regresaologistica.predict(x_teste)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_teste, y_pred)

array([[59915,     1],
       [   84,     0]], dtype=int64)

**12.** Execute o método `.score()` do modelo nos dados de teste e imprima a pontuação do teste.

A pontuação do modelo nos dados de teste processará os dados de teste por meio do modelo treinado e preverá quais transações são fraudulentas. A pontuação retornada é a porcentagem de classificações corretas, ou a precisão, e será um indicador do sucesso do seu modelo.

In [14]:
regresaologistica.score(x_teste, y_teste)

0.9985833333333334

**13.** Imprima os coeficientes do modelo para ver a importância de cada variável para a previsão. Qual característica foi mais importante? Menos importante?

In [15]:
coeficientes = regresaologistica.coef_
print(coeficientes)

[[ 0.22751718 -0.74531827  2.26232145 -0.71771267]]


#### Podemos notar que a variavel de quantia (amount) foi quem teve o maior menor na previsao, e ehMovimentacao teve o maior peso.

**14.** Vamos usar o modelo criado para processar mais transações que passaram por nossos sistemas. Existem três matrizes numpy abaixo com uma amostra com informações sobre novas transações “Novos dados de transação”

Crie uma quarta matriz, `minha_transacao` e adicione qualquer informação de transação que desejar. Certifique-se de inserir todos os valores como `floats`.

In [16]:
# Novos dados de transação

transacao1 = np.array([123456.78, 0.0, 1.0, 54670.1])
transacao2 = np.array([98765.43, 1.0, 0.0, 8524.75])
transacao3 = np.array([543678.31, 1.0, 0.0, 510025.5])

# Crie dados de uma nova transação
minha_transacao = np.array([39000.78, 0.0, 1.0, 94670.1])

**15.** Combine as novas transações e a matriz que você criou (`minha_transacao`) em uma única matriz numpy chamada `conjunto_transacoes`.

In [17]:
conjunto_transacoes = np.array([transacao1, transacao2, transacao3, minha_transacao])

**16.** Como o modelo de regressão logística foi treinado com dados padronizados, também devemos patronizar os dados sobre os quais estamos fazendo previsões. Usando o objeto `StandardScaler`  criado anteriormente, aplique o método `.transform()` na matriz `conjunto_transacoes` e salve o resultado em `conjunto_transacoes`.

In [18]:
conjunto_transacoes = ss.transform(conjunto_transacoes)

c:\Users\rafin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


**17.** Quais transações são fraudulentas? Use o método `.predict()` do seu modelo na matriz `conjunto_transacoes` e imprima o resultado para descobrir.

Quer ver as probabilidades que levaram a essas previsões? Chame o método `.predict_proba()` do seu modelo e imprima o resultado. A 1ª coluna é a probabilidade de uma transação não ser fraudulenta e a 2ª coluna é a probabilidade de uma transação ser fraudulenta (que foi calculada pelo modelo para tomar a decisão final de classificação).

In [19]:
print(regresaologistica.predict(conjunto_transacoes))

regresaologistica.predict_proba(conjunto_transacoes)

[0 0 0 0]


array([[9.96486091e-01, 3.51390904e-03],
       [9.99992324e-01, 7.67627178e-06],
       [9.99991739e-01, 8.26075115e-06],
       [9.96619247e-01, 3.38075326e-03]])